<a href="https://colab.research.google.com/github/gumendes/atividades-extensionistas-gumendes/blob/main/exploracao_dados.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# ========================================
# ANÁLISE DE DADOS - PRAÇA ZUMBI DOS PALMARES
# Jupyter Notebook Version
# ========================================

# %% [markdown]
# # 🏃‍♀️ Análise de Atividades Físicas - Praça Zumbi dos Palmares
#
# **Objetivo:** Analisar dados de presença de idosos nas atividades físicas
#
# **Período:** Agosto e Setembro 2025
#
# **Setor:** Parque América, São Paulo

# %% [markdown]
# ## 📦 1. Importar Bibliotecas

# %%
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
warnings.filterwarnings('ignore')

# Configurações de visualização
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)


# %% [markdown]
# ## 📊 2. Carregar e Preparar os Dados

# %%
# Carregar dados
df = pd.read_csv('../content/dados_praça_set-out.csv', sep=';', encoding='ISO-8859-1')

# Normalizar nomes das colunas
df.columns = df.columns.str.strip()
df = df.rename(columns={'Hor rio': 'horario', 'Nome': 'nome'})

# Corrigir nomes com caracteres especiais
df['nome'] = df['nome'].replace({
    'L£cia': 'Lúcia',
    'J£lia': 'Júlia',
    'F tima': 'Fátima'
})

# Converter tipos de dados
df['data'] = pd.to_datetime(df['data'])
df['presente'] = df['presente'].astype(bool)
df['idade'] = df['idade'].astype(int)
df['duracao_min'] = df['duracao_min'].astype(int)
df['avaliacao_aula'] = df['avaliacao_aula'].astype(int)

print(f"📊 Total de registros: {len(df)}")
print(f"📅 Período: {df['data'].min().strftime('%d/%m/%Y')} a {df['data'].max().strftime('%d/%m/%Y')}")

# Visualizar primeiras linhas
df.head()

# %% [markdown]
# ## 👥 3. Informações Sobre as Alunas

# %%
# Criar dataframe de alunas únicas
df_alunas = df[['id_aluna', 'nome', 'idade', 'sexo']].drop_duplicates().sort_values('id_aluna').reset_index(drop=True)

print(f"👥 Total de alunas cadastradas: {len(df_alunas)}")
print(f"📊 Idade média: {df_alunas['idade'].mean():.1f} anos")
print(f"📊 Idade mínima: {df_alunas['idade'].min()} anos")
print(f"📊 Idade máxima: {df_alunas['idade'].max()} anos")

# Exibir tabela de alunas
df_alunas

# %%
# Distribuição de idades
fig = px.histogram(df_alunas, x='idade', nbins=10,
                   title='Distribuição de Idades das Alunas',
                   labels={'idade': 'Idade (anos)', 'count': 'Quantidade'},
                   color_discrete_sequence=['#1f77b4'])
fig.update_layout(showlegend=False, height=400)
fig.show()

# %% [markdown]
# ## 📈 4. Análise de Presença Geral

# %%
# Estatísticas gerais
total_registros = len(df)
total_presencas = df['presente'].sum()
total_faltas = (~df['presente']).sum()
taxa_presenca_geral = (total_presencas / total_registros) * 100

print("="*60)
print("📊 ESTATÍSTICAS GERAIS DE PRESENÇA")
print("="*60)
print(f"Total de registros: {total_registros}")
print(f"Total de presenças: {total_presencas}")
print(f"Total de faltas: {total_faltas}")
print(f"Taxa de presença: {taxa_presenca_geral:.2f}%")
print("="*60)

# %%
# Evolução da presença ao longo do tempo
presenca_diaria = df.groupby('data')['presente'].agg(['sum', 'count', 'mean']).reset_index()
presenca_diaria['taxa'] = presenca_diaria['mean'] * 100

fig = go.Figure()
fig.add_trace(go.Scatter(
    x=presenca_diaria['data'],
    y=presenca_diaria['taxa'],
    mode='lines+markers',
    name='Taxa de Presença',
    line=dict(color='#1f77b4', width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title='Evolução da Taxa de Presença Diária',
    xaxis_title='Data',
    yaxis_title='Taxa de Presença (%)',
    height=500,
    hovermode='x unified'
)
fig.show()

# %% [markdown]
# ## 🏃 5. Análise por Tipo de Exercício

# %%
# Análise por tipo de exercício
exercicio_stats = df.groupby('tipo_exercicio').agg({
    'presente': ['sum', 'count', 'mean'],
    'duracao_min': 'mean',
    'avaliacao_aula': 'mean'
}).round(2)

exercicio_stats.columns = ['Presenças', 'Total Aulas', 'Taxa Presença', 'Duração Média (min)', 'Avaliação Média']
exercicio_stats['Taxa Presença'] = (exercicio_stats['Taxa Presença'] * 100).round(1)
exercicio_stats = exercicio_stats.sort_values('Taxa Presença', ascending=False)

print("="*60)
print("🏃 ANÁLISE POR TIPO DE EXERCÍCIO")
print("="*60)
print(exercicio_stats)
print("="*60)

# %%
# Gráfico: Taxa de presença por tipo de exercício
fig = px.bar(
    exercicio_stats.reset_index(),
    x='tipo_exercicio',
    y='Taxa Presença',
    title='Taxa de Presença por Tipo de Exercício',
    labels={'tipo_exercicio': 'Tipo de Exercício', 'Taxa Presença': 'Taxa de Presença (%)'},
    color='Taxa Presença',
    color_continuous_scale='Blues',
    text='Taxa Presença'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(showlegend=False, height=500)
fig.show()

# %% [markdown]
# ## 🌤️ 6. Análise por Condição Climática

# %%
# Análise por clima
clima_stats = df.groupby('clima').agg({
    'presente': ['sum', 'count', 'mean']
}).round(2)

clima_stats.columns = ['Presenças', 'Total Registros', 'Taxa Presença']
clima_stats['Taxa Presença'] = (clima_stats['Taxa Presença'] * 100).round(1)
clima_stats = clima_stats.sort_values('Taxa Presença', ascending=False)

print("="*60)
print("🌤️ ANÁLISE POR CONDIÇÃO CLIMÁTICA")
print("="*60)
print(clima_stats)
print("="*60)

# %%
# Gráfico: Impacto do clima na presença
fig = px.bar(
    clima_stats.reset_index(),
    x='clima',
    y='Taxa Presença',
    title='Impacto do Clima na Taxa de Presença',
    labels={'clima': 'Condição Climática', 'Taxa Presença': 'Taxa de Presença (%)'},
    color='Taxa Presença',
    color_continuous_scale='RdYlBu',
    text='Taxa Presença'
)
fig.update_traces(texttemplate='%{text:.1f}%', textposition='outside')
fig.update_layout(showlegend=False, height=500)
fig.show()

# %% [markdown]
# ## 📅 7. Análise por Dia da Semana

# %%
# Análise por dia da semana
ordem_dias = ['Segunda-feira', 'Terça-feira', 'Quarta-feira', 'Quinta-feira', 'Sexta-feira', 'Sábado', 'Domingo']

dia_stats = df.groupby('dia_semana').agg({
    'presente': ['sum', 'count', 'mean']
}).round(2)

dia_stats.columns = ['Presenças', 'Total Registros', 'Taxa Presença']
dia_stats['Taxa Presença'] = (dia_stats['Taxa Presença'] * 100).round(1)

# Ordenar pelos dias da semana
dia_stats = dia_stats.reindex([d for d in ordem_dias if d in dia_stats.index])

print("="*60)
print("📅 ANÁLISE POR DIA DA SEMANA")
print("="*60)
print(dia_stats)
print("="*60)

# %%
# Gráfico: Taxa de presença por dia da semana
fig = px.line(
    dia_stats.reset_index(),
    x='dia_semana',
    y='Taxa Presença',
    title='Taxa de Presença por Dia da Semana',
    labels={'dia_semana': 'Dia da Semana', 'Taxa Presença': 'Taxa de Presença (%)'},
    markers=True
)
fig.update_traces(line_color='#1f77b4', line_width=3, marker=dict(size=12))
fig.update_layout(height=500)
fig.show()

# %% [markdown]
# ## 👑 8. Ranking das Alunas

# %%
# Calcular métricas por aluna
metricas_alunas = []

for id_aluna in df['id_aluna'].unique():
    dados_aluna = df[df['id_aluna'] == id_aluna].sort_values('data')

    nome = dados_aluna['nome'].iloc[0]
    idade = dados_aluna['idade'].iloc[0]

    # Métricas gerais
    total_registros = len(dados_aluna)
    total_presencas = dados_aluna['presente'].sum()
    taxa_presenca = (total_presencas / total_registros) * 100

    # Últimos 7 e 30 dias
    ultimos_7 = dados_aluna.tail(7)
    ultimos_30 = dados_aluna.tail(30)

    presenca_7d = ultimos_7['presente'].sum()
    presenca_30d = ultimos_30['presente'].sum()
    taxa_7d = (ultimos_7['presente'].mean() * 100) if len(ultimos_7) > 0 else 0
    taxa_30d = (ultimos_30['presente'].mean() * 100) if len(ultimos_30) > 0 else 0

    # Calcular sequência atual
    sequencia = 0
    for _, row in dados_aluna.iloc[::-1].iterrows():
        if row['presente']:
            sequencia += 1
        else:
            break

    # Risco de falta
    risco = (1 - (taxa_7d/100 * 0.7 + taxa_30d/100 * 0.3)) * 100
    risco = max(0, min(100, risco))

    # Pontos (gamificação)
    pontos = total_presencas * 10 + sequencia * 5

    metricas_alunas.append({
        'ID': id_aluna,
        'Nome': nome,
        'Idade': idade,
        'Total Presenças': int(total_presencas),
        'Total Faltas': int(total_registros - total_presencas),
        'Taxa Presença (%)': round(taxa_presenca, 1),
        'Presença 7d': int(presenca_7d),
        'Taxa 7d (%)': round(taxa_7d, 1),
        'Presença 30d': int(presenca_30d),
        'Taxa 30d (%)': round(taxa_30d, 1),
        'Sequência Atual': sequencia,
        'Risco Falta (%)': round(risco, 1),
        'Pontos': int(pontos)
    })

df_metricas = pd.DataFrame(metricas_alunas)

# Ranking por pontos
df_ranking = df_metricas.sort_values('Pontos', ascending=False).reset_index(drop=True)
df_ranking.index = df_ranking.index + 1  # Começar em 1

print("="*60)
print("👑 RANKING DAS ALUNAS (TOP 10)")
print("="*60)
print(df_ranking[['Nome', 'Pontos', 'Total Presenças', 'Sequência Atual', 'Taxa Presença (%)']].head(10))
print("="*60)

# %%
# Gráfico: Top 10 alunas
top10 = df_ranking.head(10)

fig = px.bar(
    top10,
    x='Pontos',
    y='Nome',
    orientation='h',
    title='Top 10 Alunas - Ranking de Pontos',
    labels={'Pontos': 'Pontos Totais', 'Nome': 'Aluna'},
    color='Pontos',
    color_continuous_scale='Viridis',
    text='Pontos'
)
fig.update_traces(textposition='outside')
fig.update_layout(height=500, yaxis={'categoryorder':'total ascending'})
fig.show()

# %% [markdown]
# ## ⚠️ 9. Alunas em Risco (Necessitam Atenção)

# %%
# Alunas com risco alto (>60%) ou baixa frequência (<50%)
df_risco = df_metricas[
    (df_metricas['Risco Falta (%)'] > 60) |
    (df_metricas['Taxa 7d (%)'] < 50)
].sort_values('Risco Falta (%)', ascending=False)

print("="*60)
print("⚠️ ALUNAS QUE NECESSITAM ATENÇÃO")
print("="*60)
if len(df_risco) > 0:
    print(df_risco[['Nome', 'Risco Falta (%)', 'Taxa 7d (%)', 'Presença 7d', 'Sequência Atual']])
    print(f"\n💡 Total de alunas em risco: {len(df_risco)}")
else:
    print("🎉 Nenhuma aluna em situação de risco!")
print("="*60)

# %%
# Gráfico: Distribuição de risco
fig = px.histogram(
    df_metricas,
    x='Risco Falta (%)',
    nbins=20,
    title='Distribuição do Risco de Falta das Alunas',
    labels={'Risco Falta (%)': 'Risco de Falta (%)', 'count': 'Quantidade de Alunas'},
    color_discrete_sequence=['#ff6b6b']
)
fig.add_vline(x=60, line_dash="dash", line_color="red", annotation_text="Risco Alto")
fig.add_vline(x=30, line_dash="dash", line_color="orange", annotation_text="Risco Médio")
fig.update_layout(showlegend=False, height=500)
fig.show()

# %% [markdown]
# ## 📊 10. Análise de Correlações

# %%
# Preparar dados para correlação
df_corr = df[df['presente'] == True].copy()
df_corr['presente_num'] = df['presente'].astype(int)

# Calcular médias por aluna
corr_data = df.groupby('id_aluna').agg({
    'presente': 'mean',
    'idade': 'first',
    'duracao_min': 'mean',
    'avaliacao_aula': 'mean'
}).reset_index()

corr_data.columns = ['id_aluna', 'Taxa Presença', 'Idade', 'Duração Média', 'Avaliação Média']

# Matriz de correlação
corr_matrix = corr_data[['Taxa Presença', 'Idade', 'Duração Média', 'Avaliação Média']].corr()

print("="*60)
print("📊 MATRIZ DE CORRELAÇÃO")
print("="*60)
print(corr_matrix)
print("="*60)

# %%
# Gráfico: Heatmap de correlação
fig = px.imshow(
    corr_matrix,
    title='Matriz de Correlação entre Variáveis',
    labels=dict(color="Correlação"),
    x=corr_matrix.columns,
    y=corr_matrix.columns,
    color_continuous_scale='RdBu',
    aspect="auto",
    text_auto='.2f'
)
fig.update_layout(height=500)
fig.show()

# %% [markdown]
# ## 📈 11. Evolução Individual (Exemplo)

# %%
# Selecionar uma aluna para análise detalhada (TOP 1 do ranking)
aluna_exemplo = df_ranking.iloc[0]['Nome']
id_exemplo = df_ranking.iloc[0]['ID']

dados_exemplo = df[df['id_aluna'] == id_exemplo].sort_values('data').copy()
dados_exemplo['presenca_acumulada'] = dados_exemplo['presente'].cumsum()

print(f"📊 Evolução de: {aluna_exemplo} ({id_exemplo})")
print(f"Total de presenças: {dados_exemplo['presente'].sum()}")
print(f"Taxa de presença: {dados_exemplo['presente'].mean()*100:.1f}%")

# %%
# Gráfico: Evolução da aluna
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=dados_exemplo['data'],
    y=dados_exemplo['presenca_acumulada'],
    mode='lines+markers',
    name='Presenças Acumuladas',
    line=dict(color='#2ecc71', width=3),
    marker=dict(size=8)
))

fig.update_layout(
    title=f'Evolução de {aluna_exemplo} - Presenças Acumuladas',
    xaxis_title='Data',
    yaxis_title='Presenças Acumuladas',
    height=500,
    hovermode='x unified'
)
fig.show()

# %% [markdown]
# ## 💾 12. Exportar Resultados

# %%
# Exportar métricas das alunas para CSV
df_metricas.to_csv('metricas_alunas.csv', index=False, encoding='utf-8-sig')
print("✅ Arquivo 'metricas_alunas.csv' exportado com sucesso!")

# Exportar ranking
df_ranking.to_csv('ranking_alunas.csv', index=False, encoding='utf-8-sig')
print("✅ Arquivo 'ranking_alunas.csv' exportado com sucesso!")

# Exportar alunas em risco
if len(df_risco) > 0:
    df_risco.to_csv('alunas_em_risco.csv', index=False, encoding='utf-8-sig')
    print("✅ Arquivo 'alunas_em_risco.csv' exportado com sucesso!")

# %% [markdown]
# ## 📝 13. Resumo Executivo

# %%
print("="*70)
print("📊 RESUMO EXECUTIVO - PRAÇA ZUMBI DOS PALMARES")
print("="*70)
print(f"\n📅 PERÍODO: {df['data'].min().strftime('%d/%m/%Y')} a {df['data'].max().strftime('%d/%m/%Y')}")
print(f"👥 ALUNAS CADASTRADAS: {len(df_alunas)}")
print(f"📊 REGISTROS TOTAIS: {len(df)}")
print(f"\n✅ PRESENÇAS: {total_presencas} ({taxa_presenca_geral:.1f}%)")
print(f"❌ FALTAS: {total_faltas} ({100-taxa_presenca_geral:.1f}%)")
print(f"\n🏆 ALUNA DESTAQUE: {df_ranking.iloc[0]['Nome']} ({df_ranking.iloc[0]['Pontos']} pontos)")
print(f"⚠️ ALUNAS EM RISCO: {len(df_risco)}")
print(f"\n🏃 EXERCÍCIO MAIS POPULAR: {exercicio_stats.index[0]}")
print(f"🌤️ MELHOR CLIMA: {clima_stats.index[0]}")
print(f"\n📈 IDADE MÉDIA: {df_alunas['idade'].mean():.1f} anos")
print(f"📊 DURAÇÃO MÉDIA DAS AULAS: {df[df['duracao_min']>0]['duracao_min'].mean():.1f} minutos")
print(f"⭐ AVALIAÇÃO MÉDIA: {df[df['avaliacao_aula']>0]['avaliacao_aula'].mean():.1f}/5")
print("="*70)



📊 Total de registros: 520
📅 Período: 01/08/2025 a 29/09/2025
👥 Total de alunas cadastradas: 20
📊 Idade média: 63.0 anos
📊 Idade mínima: 50 anos
📊 Idade máxima: 75 anos


📊 ESTATÍSTICAS GERAIS DE PRESENÇA
Total de registros: 520
Total de presenças: 329
Total de faltas: 191
Taxa de presença: 63.27%


🏃 ANÁLISE POR TIPO DE EXERCÍCIO
                Presenças  Total Aulas  Taxa Presença  Duração Média (min)  \
tipo_exercicio                                                               
Caminhada              69          100          69.00                28.20   
Gin stica             105          160          66.00                23.97   
Jogo com bola          79          120          66.00                24.78   
Alongamento            76          140          54.00                20.79   

                Avaliação Média  
tipo_exercicio                   
Caminhada                  2.78  
Gin stica                  2.39  
Jogo com bola              2.45  
Alongamento                2.16  


🌤️ ANÁLISE POR CONDIÇÃO CLIMÁTICA
            Presenças  Total Registros  Taxa Presença
clima                                                
Ensolarado         29               40          72.00
Nublado           103              160          64.00
Frio              197              320          62.00


📅 ANÁLISE POR DIA DA SEMANA
               Presenças  Total Registros  Taxa Presença
dia_semana                                              
Segunda-feira        119              180          66.00
Quarta-feira          92              160          57.00
Sexta-feira          118              180          66.00


👑 RANKING DAS ALUNAS (TOP 10)
           Nome  Pontos  Total Presenças  Sequência Atual  Taxa Presença (%)
1         Lúcia     205               20                1              76.90
2        Teresa     200               17                6              65.40
3   Maria Lucia     200               18                4              69.20
4        Elaine     195               19                1              73.10
5        Sandra     190               19                0              73.10
6           Ana     180               18                0              69.20
7         Maria     175               17                1              65.40
8         Marta     175               17                1              65.40
9        Silvia     170               17                0              65.40
10        Neusa     170               17                0              65.40


⚠️ ALUNAS QUE NECESSITAM ATENÇÃO
        Nome  Risco Falta (%)  Taxa 7d (%)  Presença 7d  Sequência Atual
9      Ivone            63.80        28.60            2                1
13      Vera            61.50        28.60            2                1
8     F tima            55.00        42.90            3                2
12      Rosa            52.70        42.90            3                1
15  Adelaide            51.50        42.90            3                0
1        Ana            49.20        42.90            3                0

💡 Total de alunas em risco: 6


📊 MATRIZ DE CORRELAÇÃO
                 Taxa Presença  Idade  Duração Média  Avaliação Média
Taxa Presença             1.00  -0.11           0.80             0.91
Idade                    -0.11   1.00          -0.02            -0.02
Duração Média             0.80  -0.02           1.00             0.72
Avaliação Média           0.91  -0.02           0.72             1.00


📊 Evolução de: Lúcia (A007)
Total de presenças: 20
Taxa de presença: 76.9%


✅ Arquivo 'metricas_alunas.csv' exportado com sucesso!
✅ Arquivo 'ranking_alunas.csv' exportado com sucesso!
✅ Arquivo 'alunas_em_risco.csv' exportado com sucesso!
📊 RESUMO EXECUTIVO - PRAÇA ZUMBI DOS PALMARES

📅 PERÍODO: 01/08/2025 a 29/09/2025
👥 ALUNAS CADASTRADAS: 20
📊 REGISTROS TOTAIS: 520

✅ PRESENÇAS: 17 (63.3%)
❌ FALTAS: 191 (36.7%)

🏆 ALUNA DESTAQUE: Lúcia (205 pontos)
⚠️ ALUNAS EM RISCO: 6

🏃 EXERCÍCIO MAIS POPULAR: Caminhada
🌤️ MELHOR CLIMA: Ensolarado

📈 IDADE MÉDIA: 63.0 anos
📊 DURAÇÃO MÉDIA DAS AULAS: 40.1 minutos
⭐ AVALIAÇÃO MÉDIA: 4.0/5

✅ ANÁLISE COMPLETA! Todos os resultados foram gerados.
💾 Arquivos CSV exportados para análises posteriores.
📊 Gráficos interativos gerados com Plotly.

🚀 Próximo passo: Desenvolver modelo preditivo de Machine Learning!
